In [ ]:
from typing import Tuple, Callable
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras import models
from keras import regularizers
from keras.applications import MobileNetV2
from keras.engine.training import Model
import functools
import os
import numpy as np
import pickle
from keras.preprocessing.image import ImageDataGenerator
from imutils import paths
from tensorflow.keras import applications
from tensorflow.keras.applications import MobileNetV2
from keras.applications.mobilenet_v2 import preprocess_input




In [ ]:
from keras.applications.mobilenet_v2 import preprocess_input
train_dr= "/content/drive/MyDrive/hj3vvx5946-1/hj3vvx5946-1/Splited_dataset/train"
train_images= list(paths.list_images("/content/drive/MyDrive/hj3vvx5946-1/hj3vvx5946-1/Splited_dataset/train"))
num_classes=len(os.listdir(train_dr))
def get_mobilenetv2_full_tune_model_alpha_1_4_concatenated_regularised(_num_classes) \
        -> Tuple[Model, Model, int, Callable]:
    image_size = 224
    channels_count = 3
    initial_model: Model = MobileNetV2(weights='imagenet', alpha=1.4, include_top=False,
                                       input_shape=(image_size, image_size, channels_count))
    initial_model.trainable= True
    for i,layer in enumerate(initial_model.layers):
       layer.trainable=True

    initial_model_output=initial_model.output
    x=layers.GlobalAveragePooling2D()(initial_model_output)
    regularizer = regularizers.l2(0.01)
    x=layers.Dense(1024, activation='relu', kernel_regularizer=regularizer)(x)
    predictions = layers.Dense(_num_classes, activation='softmax')(x)

    model=Model(initial_model.input, predictions)
    return model, initial_model, image_size, preprocess_input


def get_callbacks_list(_early_stopping_patience, _reduce_lr_on_plateau_factor,_reduce_lr_on_plateau_patience):
    return[
        keras.callbacks.EarlyStopping(
            monitor='val_acc',
            patience=_early_stopping_patience
        ),
        keras.callbacks.ModelCheckpoint(
            verbose=1,
            filepath='best_model.h5',
            monitor='val_loss',
            save_best_only=True
        ),
        keras.callbacks.ReduceLROnPlateau(
            verbose=1,
            monitor='val_loss',
            factor=_reduce_lr_on_plateau_factor,
            patience=_reduce_lr_on_plateau_patience
        )
    ]


In [ ]:
batch_size=32
init_lr=0.01
momentum=0.9
epochs= 5
optimizer=keras.optimizers.SGD(learning_rate=init_lr, momentum=momentum)
_early_stopping_patience=10
_reduce_lr_on_plateau_factor=0.2
_reduce_lr_on_plateau_patience=3

model_function= get_mobilenetv2_full_tune_model_alpha_1_4_concatenated_regularised
result = model_function(num_classes)
model, conv_base, image_size, preprocess_function = result[:4]


model.summary()

In [ ]:
top10_acc = functools.partial(keras.metrics.top_k_categorical_accuracy, k=10)
top10_acc.__name__ = 'top10_acc'

# Compile the model:
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy', top10_acc])


In [ ]:
#data preprocessing
image_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_function,
    rotation_range=5,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    validation_split=0.1,
    fill_mode='nearest'
)
train_generator= image_datagen.flow_from_directory(
    train_dr,
    target_size=(image_size,image_size),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)
train_images_count=len(train_generator.filenames)

validation_generator = image_datagen.flow_from_directory(
    train_dr,
    target_size=(image_size, image_size),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)
validation_images_count = len(validation_generator.filenames)

with open('classes_name.pickle','wb') as handle:
  pickle.dump(validation_generator.class_indices,handle,protocol=pickle.HIGHEST_PROTOCOL)


Found 2806 images belonging to 48 classes.
Found 290 images belonging to 48 classes.


In [ ]:
train_steps = len(train_generator.filenames) // batch_size
validation_steps = len(validation_generator.filenames) // batch_size
history = model.fit_generator(
    train_generator,
    steps_per_epoch=train_steps,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_steps,
    callbacks=get_callbacks_list(_early_stopping_patience, _reduce_lr_on_plateau_factor, _reduce_lr_on_plateau_patience)
)


In [ ]:
validation_score = model.evaluate_generator(validation_generator, steps=validation_steps)
print('Validation loss: ', validation_score[0])
print('Validation acc:  ', validation_score[1])
print('Validation top 10 score:  ', validation_score[2])

In [ ]:
import keras.utils as image
from keras.models import load_model
top10_acc = functools.partial(keras.metrics.top_k_categorical_accuracy, k=10)
top10_acc.__name__ = 'top10_acc'

model = load_model('best_model.h5',custom_objects={'top10_acc': top10_acc})

with open('classes_name.pickle', 'rb') as handle:
    classes_name = pickle.load(handle)

In [ ]:
img_path='/content/drive/MyDrive/car_data/car_data/85.jpg'
image_size=224
img=image.load_img(img_path,target_size=(image_size,image_size))

x=image.img_to_array(img)
prediction=model.predict(x.reshape(1,image_size,image_size,3))

prediction_index_5=prediction.reshape(-1).argsort()[-5:][::-1]
print (np.c_[np.array(list((classes_name.keys())))[prediction_index_5],prediction.reshape(-1,)[prediction_index_5]*100])
img